# Imports

In [59]:
import requests
import json
import pandas as pd
import time
from time import sleep
import random
from random import randint
from pandas.io.json import json_normalize 

pd.set_option('display.max_columns', None)


# Extract info from website :

In [60]:
%%time

pages = []
for i in range(1,2000):

    response = requests.get(
    "https://www.vivino.com/api/explore/explore",
    params = {
        'currency_code':'EUR',
        'grape_filter':'varietal',
        'min_rating':'3.6',
        'order_by':'ratings_average',
        'order':'desc',
        'wine_type_ids[]':'4',
        'food_ids[]':'21',
        'food_ids[]':'40',
        'food_ids[]':'27',
        'food_ids[]':'38',
        'food_ids[]':'41',
        'food_ids[]':'37',
        'food_ids[]':'11',
        'food_ids[]':'39',
        'food_ids[]':'8',
        'food_ids[]':'28',
        'food_ids[]':'35',
        'food_ids[]':'34',
        'food_ids[]':'5',
        'food_ids[]':'12',
        'food_ids[]':'10',
        'food_ids[]':'20',
        'food_ids[]':'16',
        'food_ids[]':'13',
        'food_ids[]':'15',
        'food_ids[]':'9',
        'food_ids[]':'19', 
        'page':i,
        'language':'en'
    },
    headers= {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
    }
)

    #store response into "pages" list
    pages.append(response)

    # respectful nap:
    wait_time = randint(1,2000)
    sleep(wait_time/1000)

CPU times: user 18.5 s, sys: 2.04 s, total: 20.5 s
Wall time: 42min 52s


In [61]:
len(pages)

1999

In [62]:
# Nr. of wines
response.json()['explore_vintage']['records_matched']

342

In [63]:
#response.json()['explore_vintage']['matches'][0].keys()

## Vintage

In [64]:
all_wines = []
vintage = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        vintage.append(all_wines[i][j]['vintage'])

In [65]:
vintage_df = pd.DataFrame(vintage)
vintage_df.head(3)

,id,seo_name,name,statistics,image,wine,year,grapes,has_valid_ratings,top_list_rankings
0,164554740,chateau-d-esclans-garrus-rose-2020,Château d'Esclans Garrus Rosé 2020,"{'status': 'Normal', 'ratings_count': 92, 'rat...",{'location': '//images.vivino.com/thumbs/1Y51I...,"{'id': 1647837, 'name': 'Garrus Rosé', 'seo_na...",2020,None,True,NaN
1,164549061,domaines-ott-etoile-rose-bandol-2019,Domaines Ott Étoile Rosé 2019,"{'status': 'Normal', 'ratings_count': 51, 'rat...",{'location': '//images.vivino.com/thumbs/rMFJg...,"{'id': 8521982, 'name': 'Étoile Rosé', 'seo_na...",2019,None,True,NaN
2,164990951,domaines-ott-etoile-rose-bandol-2020,Domaines Ott Étoile Rosé 2020,"{'status': 'Normal', 'ratings_count': 37, 'rat...",{'location': '//images.vivino.com/thumbs/rMFJg...,"{'id': 8521982, 'name': 'Étoile Rosé', 'seo_na...",2020,None,True,NaN


In [66]:
vintage_df['id'].nunique()

342

In [67]:
#vintage_df['top_list_rankings'].value_counts(dropna = False)

#### Remove unnecessary columns

 - 'statistics' and 'wine' info will be extracted separetely, therefore I will drop these columns as well;
 - 'image' and 'top_list_rankings' are not required for the model
 - 'grapes' only has None values. This info is captured under wines section
 - 'has_valid_ratings' is always 'True'

In [68]:
vintage_df = vintage_df.drop(['statistics','image','wine','grapes','has_valid_ratings','top_list_rankings'], axis = 1)
vintage_df

,id,seo_name,name,year
0,164554740,chateau-d-esclans-garrus-rose-2020,Château d'Esclans Garrus Rosé 2020,2020
1,164549061,domaines-ott-etoile-rose-bandol-2019,Domaines Ott Étoile Rosé 2019,2019
2,164990951,domaines-ott-etoile-rose-bandol-2020,Domaines Ott Étoile Rosé 2020,2020
3,167167934,miraval-muse-de-miraval-2020,Miraval Muse de Miraval 2020,2020
4,170927155,favori-la-favorite-2021,Favori La Favorite 2021,2021
...,...,...,...,...
337,170236204,philipp-kuhn-rose-2021,Philipp Kuhn Rosé 2021,2021
338,168495692,laurent-miquel-selection-cazal-viel-rose-2021,Laurent Miquel Séléction Cazal Viel Rosé 2021,2021
339,167996090,domaine-montrose-rose-2021,Domaine Montrose Rosé 2021,2021
340,162903520,spier-signature-chardonnay-pinot-noir-2020,Spier Signature Chardonnay - Pinot Noir 2020,2020


## Statistics

In [69]:
all_wines = []
statistics = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        statistics.append(all_wines[i][j]['vintage']['statistics'])

In [70]:
statistics_df = pd.DataFrame(statistics)
statistics_df.head(3)

,status,ratings_count,ratings_average,labels_count,wine_ratings_count,wine_ratings_average,wine_status
0,Normal,92,4.6,546,3223,4.4,Normal
1,Normal,51,4.5,147,152,4.5,Normal
2,Normal,37,4.4,75,152,4.5,Normal


In [71]:
len(statistics_df['status'])

342

#### Remove unnecessary columns

In [72]:
# for col in statistics_df.columns:
#     #print(customer_df[col].unique)
#     print('Name: {}'.format(statistics_df[col].name))    # column name
#     print('Type: {}'.format(statistics_df[col].dtype))   # column type
#     print('Unique values: {}'.format(len(statistics_df[col].unique())))   # column unique values
#     print('% Unique values: {}'.format(round(len(statistics_df[col].unique())/len(statistics_df)*100,1)))  # percentage
#     print('\n----------')
    

In [73]:
statistics_df = statistics_df.drop(['status','labels_count','wine_ratings_count','wine_ratings_average','wine_status'], axis = 1)
statistics_df

,ratings_count,ratings_average
0,92,4.6
1,51,4.5
2,37,4.4
3,48,4.3
4,42,4.3
...,...,...
337,48,3.6
338,79,3.6
339,221,3.6
340,384,3.6


## Wine info

In [74]:
all_wines = []
wine = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        wine.append(all_wines[i][j]['vintage']['wine'])

In [75]:
wine_df = pd.DataFrame(wine)

wine_df.columns = 'wine_' + wine_df.columns
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings
0,1647837,Garrus Rosé,garrus-rose,4,0,False,"{'id': 450, 'name': 'Côtes de Provence', 'name...","{'id': 6905, 'name': 'Château d'Esclans', 'seo...","{'structure': {'acidity': 3.8226292, 'fizzines...","{'status': 'Normal', 'ratings_count': 3223, 'r...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True
1,8521982,Étoile Rosé,etoile-rose-bandol,4,0,False,"{'id': 648, 'name': 'Bandol', 'name_en': '', '...","{'id': 2838, 'name': 'Domaines Ott', 'seo_name...","{'structure': {'acidity': 3.65, 'fizziness': N...","{'status': 'Normal', 'ratings_count': 152, 'ra...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True
2,8521982,Étoile Rosé,etoile-rose-bandol,4,0,False,"{'id': 648, 'name': 'Bandol', 'name_en': '', '...","{'id': 2838, 'name': 'Domaines Ott', 'seo_name...","{'structure': {'acidity': 3.65, 'fizziness': N...","{'status': 'Normal', 'ratings_count': 152, 'ra...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True


### wine_region

In [76]:
#Flat columns
wine_region = pd.json_normalize(wine_df['wine_region'])
wine_region.columns = 'wine_region_' + wine_region.columns
wine_region.shape

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_region], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_region'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(342, 33)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image
0,1647837,Garrus Rosé,garrus-rose,4,0,False,"{'id': 450, 'name': 'Côtes de Provence', 'name...","{'id': 6905, 'name': 'Château d'Esclans', 'seo...","{'structure': {'acidity': 3.8226292, 'fizzines...","{'status': 'Normal', 'ratings_count': 3223, 'r...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,450.0,Côtes de Provence,,cotes-de-provence,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/f6SgQn...,//thumbs.vivino.com/region_backgrounds/f6SgQnC...,//thumbs.vivino.com/region_backgrounds/f6SgQnC...,NaN
1,8521982,Étoile Rosé,etoile-rose-bandol,4,0,False,"{'id': 648, 'name': 'Bandol', 'name_en': '', '...","{'id': 2838, 'name': 'Domaines Ott', 'seo_name...","{'structure': {'acidity': 3.65, 'fizziness': N...","{'status': 'Normal', 'ratings_count': 152, 'ra...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,648.0,Bandol,,bandol,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,NaN,NaN,NaN
2,8521982,Étoile Rosé,etoile-rose-bandol,4,0,False,"{'id': 648, 'name': 'Bandol', 'name_en': '', '...","{'id': 2838, 'name': 'Domaines Ott', 'seo_name...","{'structure': {'acidity': 3.65, 'fizziness': N...","{'status': 'Normal', 'ratings_count': 152, 'ra...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,648.0,Bandol,,bandol,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,NaN,NaN,NaN


### wine_winery

In [77]:
#Flat columns
wine_winery = pd.json_normalize(wine_df['wine_winery'])
wine_winery.columns = 'wine_winery_' + wine_winery.columns
# print(wine_winery.shape)
# wine_winery

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_winery], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(342, 42)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image
0,1647837,Garrus Rosé,garrus-rose,4,0,False,"{'id': 450, 'name': 'Côtes de Provence', 'name...","{'id': 6905, 'name': 'Château d'Esclans', 'seo...","{'structure': {'acidity': 3.8226292, 'fizzines...","{'status': 'Normal', 'ratings_count': 3223, 'r...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,450.0,Côtes de Provence,,cotes-de-provence,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/f6SgQn...,//thumbs.vivino.com/region_backgrounds/f6SgQnC...,//thumbs.vivino.com/region_backgrounds/f6SgQnC...,NaN,6905,Château d'Esclans,chateau-d-esclans,0,//images.vivino.com/wineries/backgrounds/O6R4N...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,NaN
1,8521982,Étoile Rosé,etoile-rose-bandol,4,0,False,"{'id': 648, 'name': 'Bandol', 'name_en': '', '...","{'id': 2838, 'name': 'Domaines Ott', 'seo_name...","{'structure': {'acidity': 3.65, 'fizziness': N...","{'status': 'Normal', 'ratings_count': 152, 'ra...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,648.0,Bandol,,bandol,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,NaN,NaN,NaN,2838,Domaines Ott,domaines-ott,0,NaN,NaN,NaN,NaN,NaN
2,8521982,Étoile Rosé,etoile-rose-bandol,4,0,False,"{'id': 648, 'name': 'Bandol', 'name_en': '', '...","{'id': 2838, 'name': 'Domaines Ott', 'seo_name...","{'structure': {'acidity': 3.65, 'fizziness': N...","{'status': 'Normal', 'ratings_count': 152, 'ra...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,648.0,Bandol,,bandol,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,NaN,NaN,NaN,2838,Domaines Ott,domaines-ott,0,NaN,NaN,NaN,NaN,NaN


### wine_taste

In [78]:
#Flat columns
wine_taste = pd.json_normalize(wine_df['wine_taste'])
wine_taste.columns = 'wine_taste_' + wine_taste.columns
#print(wine_taste.shape)
#wine_taste

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_taste], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(342, 51)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_flavor,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure
0,1647837,Garrus Rosé,garrus-rose,4,0,False,"{'id': 450, 'name': 'Côtes de Provence', 'name...","{'id': 6905, 'name': 'Château d'Esclans', 'seo...","{'structure': {'acidity': 3.8226292, 'fizzines...","{'status': 'Normal', 'ratings_count': 3223, 'r...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,450.0,Côtes de Provence,,cotes-de-provence,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/f6SgQn...,//thumbs.vivino.com/region_backgrounds/f6SgQnC...,//thumbs.vivino.com/region_backgrounds/f6SgQnC...,NaN,6905,Château d'Esclans,chateau-d-esclans,0,//images.vivino.com/wineries/backgrounds/O6R4N...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,NaN,"[{'group': 'oak', 'stats': {'count': 140, 'sco...",3.822629,NaN,2.722223,1.363807,NaN,80.0,201.0,NaN
1,8521982,Étoile Rosé,etoile-rose-bandol,4,0,False,"{'id': 648, 'name': 'Bandol', 'name_en': '', '...","{'id': 2838, 'name': 'Domaines Ott', 'seo_name...","{'structure': {'acidity': 3.65, 'fizziness': N...","{'status': 'Normal', 'ratings_count': 152, 'ra...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,648.0,Bandol,,bandol,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,NaN,NaN,NaN,2838,Domaines Ott,domaines-ott,0,NaN,NaN,NaN,NaN,NaN,"[{'group': 'tropical_fruit', 'stats': {'count'...",3.650000,NaN,2.691714,1.549714,NaN,10.0,3.0,NaN
2,8521982,Étoile Rosé,etoile-rose-bandol,4,0,False,"{'id': 648, 'name': 'Bandol', 'name_en': '', '...","{'id': 2838, 'name': 'Domaines Ott', 'seo_name...","{'structure': {'acidity': 3.65, 'fizziness': N...","{'status': 'Normal', 'ratings_count': 152, 'ra...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,648.0,Bandol,,bandol,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,NaN,NaN,NaN,2838,Domaines Ott,domaines-ott,0,NaN,NaN,NaN,NaN,NaN,"[{'group': 'tropical_fruit', 'stats': {'count'...",3.650000,NaN,2.691714,1.549714,NaN,10.0,3.0,NaN


#### wine_taste_flavor

In [79]:
#Flat columns
wine_flavor = pd.json_normalize(wine_df['wine_taste_flavor'])


# #Flat the first 4 columns from flavor table. This are most voted tastes by users
wine_flavor_1 = pd.json_normalize(wine_flavor[0])
wine_flavor_1.columns = 'wine_taste_flavor_1' + wine_flavor_1.columns
wine_flavor_2 = pd.json_normalize(wine_flavor[1])
wine_flavor_2.columns = 'wine_taste_flavor_2' + wine_flavor_2.columns
wine_flavor_3 = pd.json_normalize(wine_flavor[2])
wine_flavor_3.columns = 'wine_taste_flavor_3' + wine_flavor_3.columns
wine_flavor_4 = pd.json_normalize(wine_flavor[3])
wine_flavor_4.columns = 'wine_taste_flavor_4' + wine_flavor_4.columns



# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_flavor_1, wine_flavor_2, wine_flavor_3, wine_flavor_4], axis=1)
wine_df = wine_df.drop('wine_taste_flavor', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1primary_keywords,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2primary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3primary_keywords,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score
0,1647837,Garrus Rosé,garrus-rose,4,0,False,"{'id': 450, 'name': 'Côtes de Provence', 'name...","{'id': 6905, 'name': 'Château d'Esclans', 'seo...","{'structure': {'acidity': 3.8226292, 'fizzines...","{'status': 'Normal', 'ratings_count': 3223, 'r...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,450.0,Côtes de Provence,,cotes-de-provence,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/f6SgQn...,//thumbs.vivino.com/region_backgrounds/f6SgQnC...,//thumbs.vivino.com/region_backgrounds/f6SgQnC...,NaN,6905,Château d'Esclans,chateau-d-esclans,0,//images.vivino.com/wineries/backgrounds/O6R4N...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,NaN,3.822629,NaN,2.722223,1.363807,NaN,80.0,201.0,NaN,oak,"[{'id': 292, 'name': 'oak', 'count': 87}, {'id...","[{'id': 384, 'name': 'smoke', 'count': 7}, {'i...",140,16884,red_fruit,"[{'id': 400, 'name': 'strawberry', 'count': 70...",94.0,11500.0,NaN,microbio,"[{'id': 123, 'name': 'cream', 'count': 54}, {'...","[{'id': 70, 'name': 'butter', 'count': 33}, {'...",89.0,7847.0,non_oak,"[{'id': 417, 'name': 'toast', 'count': 8}, {'i...","[{'id': 434, 'name': 'vanilla', 'count': 48}, ...",73.0,1846.0
1,8521982,Étoile Rosé,etoile-rose-bandol,4,0,False,"{'id': 648, 'name': 'Bandol', 'name_en': '', '...","{'id': 2838, 'name': 'Domaines Ott', 'seo_name...","{'structure': {'acidity': 3.65, 'fizziness': N...","{'status': 'Normal', 'ratings_count': 152, 'ra...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,648.0,Bandol,,bandol,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,NaN,NaN,NaN,2838,Domaines Ott,domaines-ott,0,NaN,NaN,NaN,NaN,NaN,3.650000,NaN,2.691714,1.549714,NaN,10.0,3.0,NaN,tropical_fruit,"[{'id': 261, 'name': 

#### wine_taste_flavor_keywords

In [80]:
wine_taste_keyword1 = pd.json_normalize(wine_df['wine_taste_flavor_1primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword1_1 = pd.json_normalize(wine_taste_keyword1[0])
wine_taste_keyword1_1.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword1_1.columns
wine_taste_keyword1_2 = pd.json_normalize(wine_taste_keyword1[1])
wine_taste_keyword1_2.columns = 'wine_taste_flavor_2primary_keywords' + wine_taste_keyword1_2.columns
wine_taste_keyword1_3 = pd.json_normalize(wine_taste_keyword1[2])
wine_taste_keyword1_3.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword1_3.columns
wine_taste_keyword1_4 = pd.json_normalize(wine_taste_keyword1[3])
wine_taste_keyword1_4.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword1_4.columns
wine_taste_keyword1_5 = pd.json_normalize(wine_taste_keyword1[4])
wine_taste_keyword1_5.columns = 'wine_taste_flavor_5primary_keywords' + wine_taste_keyword1_5.columns



#combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword1_1, wine_taste_keyword1_2, wine_taste_keyword1_3, wine_taste_keyword1_4, wine_taste_keyword1_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_1primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2primary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3primary_keywords,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount
0,1647837,Garrus Rosé,garrus-rose,4,0,False,"{'id': 450, 'name': 'Côtes de Provence', 'name...","{'id': 6905, 'name': 'Château d'Esclans', 'seo...","{'structure': {'acidity': 3.8226292, 'fizzines...","{'status': 'Normal', 'ratings_count': 3223, 'r...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,450.0,Côtes de Provence,,cotes-de-provence,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/f6SgQn...,//thumbs.vivino.com/region_backgrounds/f6SgQnC...,//thumbs.vivino.com/region_backgrounds/f6SgQnC...,NaN,6905,Château d'Esclans,chateau-d-esclans,0,//images.vivino.com/wineries/backgrounds/O6R4N...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,NaN,3.822629,NaN,2.722223,1.363807,NaN,80.0,201.0,NaN,oak,"[{'id': 384, 'name': 'smoke', 'count': 7}, {'i...",140,16884,red_fruit,"[{'id': 400, 'name': 'strawberry', 'count': 70...",94.0,11500.0,NaN,microbio,"[{'id': 123, 'name': 'cream', 'count': 54}, {'...","[{'id': 70, 'name': 'butter', 'count': 33}, {'...",89.0,7847.0,non_oak,"[{'id': 417, 'name': 'toast', 'count': 8}, {'i...","[{'id': 434, 'name': 'vanilla', 'count': 48}, ...",73.0,1846.0,292.0,oak,87.0,434.0,vanilla,48.0,70.0,butter,33.0,79.0,caramel,7.0,115.0,coconut,3.0
1,8521982,Étoile Rosé,etoil

In [81]:
wine_taste_keyword2 = pd.json_normalize(wine_df['wine_taste_flavor_2primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword2_1 = pd.json_normalize(wine_taste_keyword2[0])
wine_taste_keyword2_1.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_1.columns
wine_taste_keyword2_2 = pd.json_normalize(wine_taste_keyword2[1])
wine_taste_keyword2_2.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_2.columns
wine_taste_keyword2_3 = pd.json_normalize(wine_taste_keyword2[2])
wine_taste_keyword2_3.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_3.columns
wine_taste_keyword2_4 = pd.json_normalize(wine_taste_keyword2[3])
wine_taste_keyword2_4.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_4.columns
wine_taste_keyword2_5 = pd.json_normalize(wine_taste_keyword2[4])
wine_taste_keyword2_5.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_5.columns



# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword2_1, wine_taste_keyword2_2, wine_taste_keyword2_3, wine_taste_keyword2_4, wine_taste_keyword2_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_2primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3primary_keywords,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount
0,1647837,Garrus Rosé,garrus-rose,4,0,False,"{'id': 450, 'name': 'Côtes de Provence', 'name...","{'id': 6905, 'name': 'Château d'Esclans', 'seo...","{'structure': {'acidity': 3.8226292, 'fizzines...","{'status': 'Normal', 'ratings_count': 3223, 'r...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,450.0,Côtes de Provence,,cotes-de-provence,fr,Frankrijk,France,france,EUR,Euros,€,NaN,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/f6SgQn...,//thumbs.vivino.com/region_backgrounds/f6SgQnC...,//thumbs.vivino.com/region_backgrounds/f6SgQnC...,NaN,6905,Château d'Esclans,chateau-d-esclans,0,//images.vivino.com/wineries/backgrounds/O6R4N...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,//thumbs.vivino.com/winery_backgrounds/O6R4Ncr...,Na

In [82]:
wine_taste_keyword3 = pd.json_normalize(wine_df['wine_taste_flavor_3primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword3_1 = pd.json_normalize(wine_taste_keyword2[0])
wine_taste_keyword3_1.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_1.columns
wine_taste_keyword3_2 = pd.json_normalize(wine_taste_keyword2[1])
wine_taste_keyword3_2.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_2.columns
wine_taste_keyword3_3 = pd.json_normalize(wine_taste_keyword2[2])
wine_taste_keyword3_3.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_3.columns
wine_taste_keyword3_4 = pd.json_normalize(wine_taste_keyword2[3])
wine_taste_keyword3_4.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_4.columns
wine_taste_keyword3_5 = pd.json_normalize(wine_taste_keyword2[4])
wine_taste_keyword3_5.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_5.columns



# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword3_1, wine_taste_keyword3_2, wine_taste_keyword3_3, wine_taste_keyword3_4, wine_taste_keyword3_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_3primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount
0,1647837,Garrus Rosé,garrus-rose,4,0,False,"{'id': 450, 'name': 'Côtes de Provence', 'name...","{'id': 6905, 'name': 'Château d'Esclans', 'seo...","{'structure': {'acidity': 3.8226292, 'fizzines...","{'status': 'Normal', 'ratings_count': 3223, 'r...","{'id': 173, 'seo_name': 'provence-rose', 'regi...",True,450.0,Côtes 

In [83]:
wine_taste_keyword4 = pd.json_normalize(wine_df['wine_taste_flavor_4primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword4_1 = pd.json_normalize(wine_taste_keyword2[0])
wine_taste_keyword4_1.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_1.columns
wine_taste_keyword4_2 = pd.json_normalize(wine_taste_keyword2[1])
wine_taste_keyword4_2.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_2.columns
wine_taste_keyword4_3 = pd.json_normalize(wine_taste_keyword2[2])
wine_taste_keyword4_3.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_3.columns
wine_taste_keyword4_4 = pd.json_normalize(wine_taste_keyword2[3])
wine_taste_keyword4_4.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_4.columns
wine_taste_keyword4_5 = pd.json_normalize(wine_taste_keyword2[4])
wine_taste_keyword4_5.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_5.columns


# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword4_1, wine_taste_keyword4_2, wine_taste_keyword4_3, wine_taste_keyword4_4, wine_taste_keyword4_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_4primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,

### wine_statistics

In [84]:
#Flat columns
wine_statistics = pd.json_normalize(wine_df['wine_statistics'])
wine_statistics.columns = 'wine_statistics_' + wine_statistics.columns
# print(wine_statistics.shape)
# wine_statistics

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_statistics], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(342, 131)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,

### wine_style

In [85]:
#Flat columns
wine_style = pd.json_normalize(wine_df['wine_style'])
wine_style.columns = 'wine_style_' + wine_style.columns
# print(wine_style.shape)
# wine_style

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_style], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(342, 189)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,

#### wine_style_food

In [86]:
#Flat columns
wine_food = pd.json_normalize(wine_df['wine_style_food'])

#Flat each column from food table
wine_food_1 = pd.json_normalize(wine_food[0])
wine_food_1.columns = 'wine_food_' + wine_food_1.columns
wine_food_2 = pd.json_normalize(wine_food[1])
wine_food_2.columns = 'wine_food_' + wine_food_2.columns
wine_food_3 = pd.json_normalize(wine_food[2])
wine_food_3.columns = 'wine_food_' + wine_food_3.columns
wine_food_4 = pd.json_normalize(wine_food[3])
wine_food_4.columns = 'wine_food_' + wine_food_4.columns
wine_food_5 = pd.json_normalize(wine_food[4])
wine_food_5.columns = 'wine_food_' + wine_food_5.columns


# #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_food_1, wine_food_2, wine_food_3, wine_food_4, wine_food_5], axis=1)
wine_df = wine_df.drop('wine_style_food', axis=1)
wine_df.shape

(342, 213)

#### wine_style_grapes

In [89]:
#Flat columns
wine_grapes = pd.json_normalize(wine_df['wine_style_grapes'])

# #Flat each column from food table
wine_grapes_1 = pd.json_normalize(wine_grapes[0])
wine_grapes_1.columns = 'wine_style_grapes' + wine_grapes_1.columns
wine_grapes_2 = pd.json_normalize(wine_grapes[1])
wine_grapes_2.columns = 'wine_style_grapes_' + wine_grapes_2.columns
wine_grapes_3 = pd.json_normalize(wine_grapes[2])
wine_grapes_3.columns = 'wine_style_grapes_' + wine_grapes_3.columns
# wine_grapes_4 = pd.json_normalize(wine_grapes[3])
# wine_grapes_4.columns = 'wine_style_grapes_' + wine_grapes_4.columns
# wine_grapes_5 = pd.json_normalize(wine_grapes[4])
# wine_grapes_5.columns = 'wine_style_grapes_' + wine_grapes_5.columns
# wine_grapes_6 = pd.json_normalize(wine_grapes[5])
# wine_grapes_6.columns = 'wine_style_grapes_' + wine_grapes_6.columns


# # #combine all dfs. drop original column, keep flattened
#wine_df = pd.concat([wine_df, wine_grapes_1, wine_grapes_2, wine_grapes_3, wine_grapes_4, wine_grapes_5, wine_grapes_6], axis=1)
wine_df = pd.concat([wine_df, wine_grapes_1, wine_grapes_2, wine_grapes_3], axis=1)
wine_df = wine_df.drop('wine_style_grapes', axis=1)
wine_df.shape

(342, 227)

## Price

In [90]:
all_wines = []
prices = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        prices.append(all_wines[i][j]['price'])

In [91]:
prices_df = pd.DataFrame(prices)
prices_df.head(3)

,id,amount,discounted_from,discount_percent,type,sku,url,visibility,bottle_type_id,currency,bottle_type
0,25568608,150.94,None,None,vc,25652,http://cultwijnen.nl/,1,1,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'...","{'id': 1, 'name': 'Fles (0.75 l)', 'short_name..."
1,22396120,109.95,None,None,vc,W111362617,https://www.wijnig.nl/domaines-ott-etoile-rose...,1,1,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'...","{'id': 1, 'name': 'Fles (0.75 l)', 'short_name..."
2,26029029,99.95,None,None,vc,OTT-20216,https://www.grandcruwijnen.nl/2020-domaines-ot...,1,1,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'...","{'id': 1, 'name': 'Fles (0.75 l)', 'short_name..."


#### Remove unnecessary columns

Price information will be used to improve recommendatio. Only the price per bottle is relevant

In [92]:
drop_lst = ['id','type','discounted_from','discount_percent','sku','url','visibility','bottle_type_id','bottle_type']
prices_df = prices_df.drop(drop_lst, axis = 1)
prices_df

,amount,currency
0,150.94,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
1,109.95,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
2,99.95,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
3,303.40,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
4,14.95,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
...,...,...
337,9.50,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
338,7.25,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
339,6.50,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
340,7.50,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."


In [93]:
#Flat columns
currency = pd.json_normalize(prices_df['currency'])

# # #combine all dfs. drop original column, keep flattened
prices_df = pd.concat([prices_df, currency], axis=1)
prices_df = prices_df.drop('currency', axis=1)
prices_df.shape

# keep only currency code
drop_lst = ['name','prefix','suffix']
prices_df = prices_df.drop(drop_lst, axis = 1)
prices_df

,amount,code
0,150.94,EUR
1,109.95,EUR
2,99.95,EUR
3,303.40,EUR
4,14.95,EUR
...,...,...
337,9.50,EUR
338,7.25,EUR
339,6.50,EUR
340,7.50,EUR


## Make full df from all tables above

In [94]:
all_wines_df = pd.concat([vintage_df,prices_df,statistics_df,wine_df], axis = 1)
print(all_wines_df.shape)
all_wines_df.head(3)

(342, 235)


,id,seo_name,name,year,amount,code,ratings_count,ratings_average,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor

In [95]:
len(all_wines_df)

342

In [96]:
columns_to_drop = ['wine_region','wine_winery','wine_taste','wine_statistics','wine_style']

In [97]:
all_wines_df = all_wines_df.drop(columns_to_drop, axis = 1)
all_wines_df

,id,seo_name,name,year,amount,code,ratings_count,ratings_average,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscoun

In [98]:
all_wines_df.to_csv('all_wines_df_type4_v3.csv')